In [26]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

## JD Sports Proximity

#### Business Question
Which TM/TS stores do not have a JD sports within 2km radius?

#### Methodology

Import cleaned data from 'JD Sports Proximity Data' notebook. This data is sourced from 'Smart' Branch Data and Foursquare, for more details see [here](https://github.com/ninjananjo/Clothing-Store-Competition/blob/master/JD%20Sports%20Proximity/JD%20Sports%20Proximity%20Data.ipynb "JD Sports Proximity Data"). Then identify closest JD Sports

In [1]:
%%capture
!pip install pandas
!pip install folium
#!conda install -c conda-forge folium --yes

import pandas as pd
import numpy as np
import folium

In [6]:
#Imports cleaned data from 'JD Sports Proximity Data' notebook

jdsports = pd.read_csv('jdsports_proximity.csv')
#if jdsports.columns[0] == 'Unnamed: 0': jdsports.drop(columns='Unnamed: 0', inplace=True)
#print(jdsports.shape)
jdsports.head()

,Host ID,Host Lat,Host Lng,TOWN,NAME,COUNTRY,Address1,Postcode,PNO,NSF,Branch IDs,Venue ID,Venue Name,Venue Distance,Venue Address,Venue Lat,Venue Lng,Venue Category
0,TM7157,53.541672,-2.112246,OLDHAM,OLDHAM,ENG,Unit 3,OL1 1XE,1361,2200,TM7157,4bf93e604a67c928dd8026cf,JD Sports,132,['Units 16-17 Lower Mall The Spindles Sc (Lowe...,53.541413,-2.114194,Sporting Goods Shop
1,TM7157,53.541672,-2.112246,OLDHAM,OLDHAM,ENG,Unit 3,OL1 1XE,1361,2200,TM7157,5b7f0f5e4a7aae002cf533ea,JD Sports,7061,"['The Sharp Project (Thorpe Rd)', 'Manchester'...",53.501436,-2.194744,Sporting Goods Shop
2,TM7157,53.541672,-2.112246,OLDHAM,OLDHAM,ENG,Unit 3,OL1 1XE,1361,2200,TM7157,5a8ecd550881587526bae41b,JD Sports,9173,"['Unit 18 Crown Point North Denton', 'Denton',...",53.459318,-2.117344,Sporting Goods Shop
3,TM7157,53.541672,-2.112246,OLDHAM,OLDHAM,ENG,Unit 3,OL1 1XE,1361,2200,TM7157,4c9dee1f46978cfa9f219c7f,JD Sports,5919,"['Units 28 & 29, The Arcades Shopping Centre (...",53.489489,-2.095046,Sporting Goods Shop
4,TM7157,53.541672,-2.112246,OLDHAM,OLDHAM,ENG,Unit 3,OL1 1XE,1361,2200,TM7157,4c6aca2f897b1b8d72b1b217,JD Sports,2522,"['Unit 3 Centre Retail Park Elk Mill', 'Oldham...",53.558100,-2.138509,Sporting Goods Shop


In [29]:
#Finds closest JD Sport to each store

closest_jd = jdsports.groupby(['Host ID'], as_index =False).min()
#print(closest_jd.shape)
#closest_jd.head()

#### Results
The map below shows TM/TS stores (blue tags) with surrounding JD Sports (black dots).
A green circle around a TM/TS store shows that there is a JD Sports within the 2km circle, a red circle is used if there is not.

In [30]:
#Maps TM/TS and JD Sports that are within 10km.
#A green circle (2km radius) is place round the TM/TS store if there is a JD Sport within the circle, a red circle is used if there is not.

m = folium.Map(location=[53.4626, -2.34612], zoom_start=6)

for branch, branch_lat, branch_lng, proximity in zip(closest_jd['Host ID'], closest_jd['Host Lat'], closest_jd['Host Lng'], closest_jd['Venue Distance']):
    if proximity < 2000:
        colour = 'green'
    else: 
        colour = 'red'
    folium.Marker(location=[branch_lat, branch_lng], popup=branch).add_to(m)
    folium.Circle(location=[branch_lat, branch_lng], radius=2000, fill=False, color=colour).add_to(m)

for branch_lat, branch_lng, jd_lat, jd_lng in zip(jdsports['Host Lat'], jdsports['Host Lng'], jdsports['Venue Lat'], jdsports['Venue Lng']):
    folium.Circle(location=[jd_lat, jd_lng], popup='JD Sports', radius=50, fill=True, color='black').add_to(m)

m

The table below shows TM/TS stores without a JD Sports within a 2km radius. These are shown in the map above with a red circle. The table is outputed to a CSV.

In [36]:
#Shows TM/TS stores without a JD Sports within a 2km radius.
closest_jd = closest_jd[closest_jd['Venue Distance']>2000]
closest_jd

,Host ID,Host Lat,Host Lng,TOWN,NAME,COUNTRY,Address1,Postcode,PNO,NSF,Branch IDs,Venue ID,Venue Name,Venue Distance,Venue Address,Venue Lat,Venue Lng,Venue Category
0,TM7157,53.541672,-2.112246,OLDHAM,OLDHAM,ENG,Unit 3,OL1 1XE,1361,2200,TM7157,4b7ec528f964a52010fe2fe3,JD Sports,2522,"['16-18 Arndale Centre, Barbarolli Mall', 'Man...",53.459318,-2.242812,Sporting Goods Shop
2,TM7782,51.520023,-0.074251,SHOREDITCH,SHOREDITCH,ENG,98 Commercial Street,E1 6LZ,968,1150,TM7782,4ac518f1f964a520ecad20e3,JD Sports,4555,"['197-203 Oxford Street', 'West End', 'Greater...",51.515239,-0.139567,Sporting Goods Shop
7,TS0003,51.574492,-0.221565,BRENT CROSS,BRENT CROSS,ENG,"Unit 72, Brent Cross Shopping Centre",NW4 3FP,780,11237,"TM7218,TS0003",4ac518f1f964a520ecad20e3,JD Sports,4334,"['197-203 Oxford Street', 'West End', 'Greater...",51.515239,-0.280536,Sporting Goods Shop
11,TS0014,51.461030,-0.304219,RICHMOND,RICHMOND,ENG,53/54 George Street,TW9 1HJ,640,2465,"TM7561,TS0014",4c0f573a2466a5931e797a21,JD Sports,5614,"['20 Church Street', 'Kingston', 'Greater Lond...",51.410596,-0.304978,Sporting Goods Shop
23,TS0062,51.751444,-0.337992,ST ALBANS,ST ALBANS,ENG,Unit 15,AL1 3HL,1602,4803,"TM7758,TS0062",4c39a9973849c928aaa1c1b1,JD Sports,6825,"['Hatfield Park Plaza, Comet Way (Comet Way)',...",51.749470,-0.471251,Sporting Goods Shop
24,TS0063,52.291022,-1.534741,LEAMINGTON SPA,LEAMINGTON SPA,ENG,4 Upper Mallstreet,CV32 4XT,1875,4300,"TM7193,TS0063",4d9dd22f5c33a35d530b0aa1,JD Sports,3446,['United Kingdom'],52.320816,-1.548500,Sporting Goods Shop
28,TS0080,49.206646,-2.198538,ST HELIER,JERSEY,JER,40 King Street,JE2 4WE,6044,1545,"TM7182,TS0080",4c7f9c0cd51ea143b1005b9f,JD Sports,7073,"['42 King St', 'St Helier', 'JE2 4WE', 'Jersey']",49.184662,-2.107299,Sporting Goods Shop
57,TS0185,-33.648333,150.734444,LONDONDERRY,LONDONDERRY,NI,Unit M7,BT48 6XY,426,4735,"TM8105,TS0185",5b7f3ce50457b7002caf4c21,JD Sports,12103,"['316 Level 2 Westfield Penrith, 585 High Stre...",-33.751662,150.693781,Sporting Goods Shop
58,TS0191,53.488485,-2.094246,ASHTON UNDER LYNE,ASHTON UNDER LYNE,ENG,51 Warrington Street,OL6 7JG,1540,1998,"TM7262,TS0191",4b7ec528f964a52010fe2fe3,JD Sports,6809,"['16-18 Arndale Centre, Barbarolli Mall', 'Man...",53.410218,-2.242812,Sporting Goods Shop
62,TS0309,51.466615,-0.096800,CANARY WHARF,CANARY WHARF,ENG,Topshop Unit 3 Canada Square,E14 5AB,497,2183,"TM8634,TS0309",4ac518f1f964a520ecad20e3,JD Sports,6171,"['197-203 Oxford Street', 'West End', 'Greater...",51.428388,-0.167446,Sporting Goods Shop


In [37]:
#Output to csv
closest_jd.to_csv('jdsports_proximity_outside2km.csv', index=False)